In [3]:
import ee 
import geemap
import numpy as np
import pandas as pd 
from matplotlib import pyplot as plt

ee.Authenticate()
ee.Initialize(project="cloud-cover-421214")


Successfully saved authorization token.


In [4]:
cushing_ok_site = [[-96.76487361449992,35.919767693818464],
[-96.7147484924296,35.919767693818464],
[-96.7147484924296,35.954792502408324],
[-96.76487361449992,35.954792502408324],
[-96.76487361449992,35.919767693818464]]

roi = ee.Geometry.Polygon(cushing_ok_site)
start_date = "2020-01-01"
end_date = "2022-01-01"

In [27]:
# Display ROI

m = geemap.Map()
m.add_basemap()
m.add_ee_layer(roi, {'color': 'red'}, 'ROI')
m.center_object(roi, zoom=13)
m

Map(center=[35.937281409522136, -96.7398110534637], controls=(WidgetControl(options=['position', 'transparent_…

In [22]:
cloudscores = ee.ImageCollection("GOOGLE/CLOUD_SCORE_PLUS/V1/S2_HARMONIZED").filterBounds(roi).filterDate(start_date, end_date)
sentinel2 = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED").filterBounds(roi).filterDate(start_date, end_date)
print(f"Cloudscore dataset size: {cloudscores.size().getInfo()}\nSentinel-2 dataset size: {sentinel2.size().getInfo()}")


Cloudscore dataset size: 294
Sentinel-2 dataset size: 294


Sanity check: visualize first day of each series to inspect cloud match. Purple = cloudier. 

In [24]:
s2_params = {'min': 0, 'max': 4000, 'bands': ['B8', 'B4', 'B3']}
cs_params = {'min': 0, 'max': 1, 'bands': ['cs'], 'palette': 'plasma'}
m.add_ee_layer(sentinel2.first(), s2_params, "Sentinel-2")
m.add_ee_layer(cloudscores.first(), cs_params, "Cloud score")
m

Map(bottom=206321.0, center=[35.92186429147193, -96.73152923583986], controls=(WidgetControl(options=['positio…

In [30]:
cl = cloudscores.toList(3)

data = {}

for i in range(cl.size().getInfo()):
    image = ee.Image(cl.get(i))
    t = ee.Date(image.get('system:time_start')).format().getInfo()
    arr = image.sampleRectangle(roi).get('cs')
    arr = np.array(arr.getInfo())
    data[t] = arr
    print(f"Array shape {arr.shape} collected on {t}")

print(data)

Array shape (400, 462) collected on 2020-01-05T17:23:51
Array shape (400, 462) collected on 2020-01-05T17:23:47
Array shape (400, 462) collected on 2020-01-10T17:23:51
{'2020-01-05T17:23:51': array([[0.89803922, 0.90196078, 0.90196078, ..., 0.88235294, 0.88235294,
        0.87843137],
       [0.89803922, 0.89803922, 0.90196078, ..., 0.87843137, 0.88235294,
        0.88235294],
       [0.89803922, 0.89803922, 0.89803922, ..., 0.87843137, 0.88235294,
        0.88235294],
       ...,
       [0.88627451, 0.89019608, 0.88627451, ..., 0.87058824, 0.87058824,
        0.86666667],
       [0.88627451, 0.89019608, 0.88627451, ..., 0.87058824, 0.87058824,
        0.86666667],
       [0.89411765, 0.89411765, 0.89411765, ..., 0.86666667, 0.86666667,
        0.86666667]]), '2020-01-05T17:23:47': array([[0.90196078, 0.90196078, 0.90196078, ..., 0.89019608, 0.89019608,
        0.89411765],
       [0.90196078, 0.90196078, 0.90196078, ..., 0.89019608, 0.89019608,
        0.89411765],
       [0.89803922,